In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from IPython.core.display import display, HTML
from IPython.lib.pretty import pprint
from itertools import chain, tee

In [3]:
from mockdown.view import View
from mockdown.display.view import display_view

In [4]:
sample_view = View("p", (0, 0, 100, 100), children=[
    View("a1", (10, 10, 45, 90), children=[
        View("b11", (20, 20, 35, 45)),
        View("b12", (20, 55, 35, 80))
    ]),
    View("a2", (55, 10, 90, 90), children=[
        View("b21", (65, 20, 80, 45)),
        View("b22", (65, 55, 80, 80))
    ])
])

In [5]:
html_str = str(display_view(sample_view, scale=3))
display(HTML(html_str))

In [6]:
from mockdown.visibility import visible_pairs

edge_pairs = visible_pairs(sample_view, deep=True)
anchor_pairs = [(e1.anchor, e2.anchor) for (e1, e2) in edge_pairs]

In [7]:
html_str = str(display_view(sample_view, edge_pairs, scale=3))
display(HTML(html_str))

In [8]:
from mockdown.logic import constrainable_pairs
pairs = list(constrainable_pairs(sample_view, anchor_pairs))

spacings = list(filter(lambda p: p.kind == 'spacing', pairs))
alignments = list(filter(lambda p: p.kind == 'alignment', pairs))

In [9]:
from mockdown.daikon import generate_decls, generate_dtrace

decls_text = generate_decls("sample_view", sample_view, spacings)
dtrace_text = generate_dtrace("sample_view", sample_view, spacings)

print(decls_text[0:150] + "\n...\n")
print(dtrace_text[0:80] + "\n...\n")

decl-version 2.0
input-language data

ppt sample_view.data:::POINT
ppt-type point
  variable a1.right_0
    var-kind variable
    dec-type spacing
   
...

sample_view.data:::POINT
a1.right_0
45
1
a2.left_0
55
1
b11.bottom_1
45
1
b12.to
...



In [12]:
DAIKONJAR="/usr/local/opt/daikon/daikon-src/daikon.jar"
INVARIANTJAR="/Users/dylan/Projects/Pipeline/MockdownInvariants/out/artifacts/mockdown_inv/mockdown-inv.jar"
LIBJARS="/Users/dylan/Projects/Pipeline/MockdownInvariants/lib/*"

from asyncio import subprocess
from tempfile import NamedTemporaryFile
from shlex import quote
import platform

def mk_tmp(suffix: str):
    return NamedTemporaryFile(mode='w+', 
                              prefix="mockdown-",
                              suffix=suffix)

with mk_tmp('.decls') as decls_file, mk_tmp('.dtrace') as dtrace_file:
    decls_file.write(decls_text)
    decls_file.flush()
    
    dtrace_file.write(dtrace_text)
    dtrace_file.flush()

    cmd = (
        f"java -cp {DAIKONJAR}:{INVARIANTJAR}:{LIBJARS} daikon.Daikon"
        " --nohierarchy"
        " --disable-all-invariants"
        " --user-defined-invariant edu.ucsd.mockdown.inv.PositionSpacingUpper"
        " --config_option edu.ucsd.mockdown.inv.PositionSpacingUpper.enabled=true"
        f" {quote(decls_file.name)}"
        f" {quote(dtrace_file.name)}"
        "-o /dev/null"  # don't write a basename.inv.gz
    )
    
    print(f"> {cmd}\n")
    
    proc = await subprocess.create_subprocess_shell(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    await proc.wait()
    stdout, stderr = await proc.communicate()

    print(stdout.decode())

> java -cp /usr/local/opt/daikon/daikon-src/daikon.jar:/Users/dylan/Projects/Pipeline/MockdownInvariants/out/artifacts/mockdown_inv/mockdown-inv.jar:/Users/dylan/Projects/Pipeline/MockdownInvariants/lib/* daikon.Daikon --nohierarchy --disable-all-invariants --user-defined-invariant edu.ucsd.mockdown.inv.PositionSpacingUpper --config_option edu.ucsd.mockdown.inv.PositionSpacingUpper.enabled=true /var/folders/nd/jzx57n691vddfstcxf4kf8c80000gn/T/mockdown-xs3a_k5b.decls /var/folders/nd/jzx57n691vddfstcxf4kf8c80000gn/T/mockdown-l8qny0yx.dtrace-o /dev/null


